<center>
  <h2>Trabajo Laboratorio de Implementación III - MCD Virtual Cohorte 2022</h2>
  <h3>Forecasting para una Empresa de Consumo Masivo</h3>
  <h4>Entrenamiento de predictor</h4>
</center>

In [4]:
# Importación librerias

# !pip install awswrangler
# !pip install boto3
import boto3
import awswrangler as wr
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
warnings.filterwarnings("ignore")

In [14]:
# Definición fechas

date_start=datetime.strptime('2017-01-01', '%Y-%m-%d')
date_end=datetime.strptime('2019-12-01', '%Y-%m-%d')
forecast_length = 2
date_future=date_end+relativedelta(months=forecast_length)
years=range(2018, 2020)

print(f"Fecha inicio historia {date_start}")
print(f"Fecha fin historia {date_end}")
print(f"Fecha futura (related time series) {date_future}")

timestamp_format="yyyy-MM-dd"
dataset_frequency='M'

# Versión modelo
data_version = '1'
grouped_cols = ["product_id"]
grouped_cols_name = "product"

Fecha inicio historia 2017-01-01 00:00:00
Fecha fin historia 2019-12-01 00:00:00
Fecha futura (related time series) 2020-02-01 00:00:00


In [6]:
# Conexiones AWS

account_id=boto3.client('sts').get_caller_identity().get('Account')
session = boto3.Session() 
forecast = session.client(service_name='forecast')
region = forecast.meta.region_name
forecastquery = session.client(service_name='forecastquery')
role = f'arn:aws:iam::{account_id}:role/ForecastRole'

In [9]:
# Qué valores predecir: media o percentiles
quantiles=['mean', '0.5']
# Horizonte de predicción
forecast_length = 2
dataset_group_arn=f'arn:aws:forecast:us-east-1:{account_id}:dataset-group/modelo_{data_version}_{grouped_cols_name}'

In [11]:
# Verificar comunicación con Amazon Forecast
assert forecast.list_predictors()

In [12]:
predictor_name_automl = f"modelo_{data_version}_predictor"

El siguiente código es para crear el predictor. Obtenemos error porque fue creado desde la UI para este primer modelo.

In [18]:
response = forecast.create_auto_predictor(
    PredictorName=predictor_name_automl,
    ForecastHorizon=forecast_length,
    ForecastTypes=quantiles,
    # ForecastDimensions=grouped_cols,
    ForecastFrequency=dataset_frequency,
    DataConfig={
        'DatasetGroupArn': dataset_group_arn,
        'AttributeConfigs': [
		{
		"AttributeName": "target_value",
		"Transformations": {
			"aggregation": "sum",
			"backfill": "zero",
			"frontfill": "none",
			"middlefill": "zero"
		}
		}
	]}, 
    TimeAlignmentBoundary={
        'DayOfMonth': 1
    },
    OptimizationMetric='MAPE',
    ExplainPredictor=False
)

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateAutoPredictor operation: A predictor with arn arn:aws:forecast:us-east-1:637423651905:predictor/modelo_1_predictor_01HZG49YKVPFHRYBPFZPG3KTYK already exists with the name modelo_1_predictor. Provide an unique name for the new predictor

In [ ]:
automl_predictor_arn_wape = response['PredictorArn']

In [ ]:
print(f"Waiting for Predictor with ARN {automl_predictor_arn_wape} to become ACTIVE. \nCurrent Status:")
forecast.describe_auto_predictor(PredictorArn=automl_predictor_arn_wape)['Status']